In [2]:
################ Code from the 'Optimization of molecular geometries' tutorial##################
# imports and relevant defines:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

# Simulation starting parameters:
symbols = ["O", "H", "H"]

# this is selected from the tutorial 'Building molecular Hamiltonians', which has the same nulcear-coordinats for water
x = np.array([ 0.        ,  0.7581    , -0.5086    , -1.49139892,  0.29819887,
        -1.2393812 , -0.68850404,  2.16155953,  0.21671681], requires_grad=True)

# these parameters are used to match up with the parameters for the VQE run on water in the paper: https://arxiv.org/pdf/2106.13840.pdf
active_electrons = 8
active_orbitals = 6

# define the hamiltonian needed to compute cost-function
def H(x):
    return qchem.molecular_hamiltonian(
        symbols,
        x,
        charge = 0,
        mult=1,
        active_electrons = active_electrons,
        active_orbitals = active_orbitals)[0]



################ Code from the 'Building the adaptive circuit' tutorial below:################

geometry = np.array([ 0.        ,  0.7581    , -0.5086    , -1.49139892,  0.29819887,
        -1.2393812 , -0.68850404,  2.16155953,  0.21671681], requires_grad=True)
active_electrons = 8
active_orbitals = 6 

H, qubits = qchem.molecular_hamiltonian(symbols, geometry, charge = 0, mult=1, active_electrons=active_electrons, active_orbitals=active_orbitals)
singles, doubles = qchem.excitations(active_electrons, qubits)

hf_state = qchem.hf_state(active_electrons, qubits)



#compute the significant double-excitation gates:
def circuit_1(params, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_1, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(doubles)
grads = circuit_gradient(params, excitations=doubles)
print("Computed gradients for all possible Double Excitation Gates: \n")
for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")   
doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected double-excitation gates: ", len(doubles_select))



# optimizing the parameters for the double-excitation gates for Ansatz-wavefunction construction
opt = qml.GradientDescentOptimizer(stepsize=0.5)
params_doubles = np.zeros(len(doubles_select), requires_grad=True)
for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)
print("Done!")



#compute the significant single-excitation gates:
def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)


cost_fn = qml.QNode(circuit_2, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)
grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)
print("Computed gradients for all possible Single Excitation Gates: \n")
for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected single-excitation gates: ", len(singles_select))

#Total Number of Gates selected to construct the Quantum Ansatz:
print("Total selected gates: "+  str(len(doubles_select) + len(singles_select)))

################# End code from the 'Building the adaptive circuit' tutorial ##################

################ Resume from the 'Optimization of molecular geometries' tutorial below: ################
hf = hf_state


#Construct optimized parametrized circuit that will be used to run the optimized VQE algorithm
# using the selected single and double excitation gates:
num_wires = qubits
dev = qml.device("lightning.qubit", wires=num_wires)
@qml.qnode(dev, interface="autograd")
def circuit(params, obs, wires):
    # prepares reference state
    qml.BasisState(hf, wires=wires)
    
    # apply all single excitations
    for i, singles in enumerate(singles_select):
        qml.SingleExcitation(params[i], wires=singles)
        
    # apply all double excitations
    for j, doubles in enumerate(doubles_select):
        qml.DoubleExcitation(params[j + len(singles_select)], wires=doubles)
                             
    # returns expectation value of the ansatz prepared from this quantum circuit:   
    return qml.expval(obs)

def cost(params, x):
    hamiltonian = H(x)
    return circuit(params, obs=hamiltonian, wires=range(num_wires)) 
    

# This function computes the finite-difference of the Hamiltonian itself with respect to each of the nuclear-coordinates
# itself, using a central-difference approximation.
def finite_diff(f, x, delta=0.01):
    """Compute the central-difference finite difference of a function"""
    gradient = []
    for i in range(len(x)):
        shift = np.zeros_like(x)
        shift[i] += 0.5 * delta
        res = (f(x + shift) - f(x - shift)) * delta**-1 # dH/dx            
        gradient.append(res)

    return gradient

def grad_x(params, x): #evaluate the expectation of the gradient components (each of the nuclear components of the Hamiltonian)
    grad_h = finite_diff(H, x)
    grad = [circuit(params, obs=obs, wires=range(num_wires)) for obs in grad_h]
    return np.array(grad)



# initialize optimizers:
opt_theta = qml.GradientDescentOptimizer(stepsize=0.2)
opt_x = qml.AdamOptimizer(stepsize=0.01)


# functions to keep track of convergence of O-H1 and O-H2 bond angle during water's geometry optimization
import math
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return (np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180/math.pi) #convert to degrees


# Re-initialize all relevant parameters to be optimized:
# nuclear-coordinate parameters:
symbols = ["O", "H", "H"]
x = np.array([ 0.        ,  0.7581    , -0.5086    , -1.49139892,  0.29819887,
        -1.2393812 , -0.68850404,  2.16155953,  0.21671681], requires_grad=True)

# circuit parameters:
theta = np.array([0.0] * (len(doubles_select) + len(singles_select)), requires_grad=True)

print("Reinitialized gradient descent parameters!")




# re-define the hamiltonian needed to compute cost-function (otherwise it gives error: Hamiltonian function not callable
# when I run the actual gradient descent code below)
def H(x):
    return qchem.molecular_hamiltonian(
        symbols,
        x,
        charge = 0,
        mult=1,
        active_electrons = active_electrons,
        active_orbitals = active_orbitals)[0]




# ~~~~~~~~~~~~~~ Final block: Calculating water's molecular geometry ~~~~~~~~~~~~~~~
from functools import partial

# store the values of the cost function
energy = []

# store the values of the first OH-bond length
bond_length = []

#store the value of the bond_angle:
bond_angle = []

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903


# Calculate starting water-molecule parameters:
OH1 = x[0:3] - x[3:6]
OH2 = x[0:3] - x[6:9]
print("Starting bond length: " + str(np.linalg.norm(x[0:3] - x[3:6]) * bohr_angs) + " Å")
print("Starting bond angle: " + str(angle_between(OH1, OH2)) + '\u00b0')
print("")

'''
Errors (and how to fix):

If error 'Hamiltonian object is not callable', re-run cell that defines H(x) all the way at the top.
If the starting bond length and angle are off, re-run cell that defines symbols and geometry for x all the way at the top
'''

eps = 1e-05
n = 0

start = time.time()
while True:

    print("Starting step " + str(n) + ":")
    print("Calculating circuit parameters gradient")
    # Optimize the circuit parameters
    theta.requires_grad = True
    x.requires_grad = False
    theta, _ = opt_theta.step(cost, theta, x) # compare with final block of code in 'Adapt-VQE' tutorial on Pennylane

    print("Calculating nuclear coordinates parameters gradient")
    # Optimize the nuclear coordinates
    theta.requires_grad = False
    x.requires_grad = True
    _, x = opt_x.step(cost, theta, x, grad_fn=grad_x)
    print("Finished calculating nuclear coordinates parameters gradient")
    

    energy.append(cost(theta, x))
    bond_length.append(np.linalg.norm(x[0:3] - x[3:6]) * bohr_angs)
    
    #calculate bond-angle between both OH-bonds
    OH1 = x[0:3] - x[3:6]
    OH2 = x[0:3] - x[6:9]
    bond_angle.append(angle_between(OH1, OH2))

    if n % 1 == 0:
        print(f"Step = {n},  E = {energy[-1]:.8f} Ha,  bond length = {bond_length[-1]:.5f} Å, bond angle = {bond_angle[-1]:.5f}" + '\u00b0')

    n += 1
    # Check maximum component of the nuclear gradient:
    max_grad = np.max(grad_x(theta, x))
    print("Max grad:", max_grad)
    print("")
    
    if n <= 1:
        continue
    
    if max_grad <= 1e-05 or np.abs(energy[-2]-energy[-1]) < eps:
        break
        

print("Total time:", time.time()-start, "seconds")

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" "Ground-state equilibrium geometry")
print("%s %4s %8s %8s" % ("symbol", "x", "y", "z"))
for i, atom in enumerate(symbols):
    print(f"  {atom}    {x[3 * i]:.4f}   {x[3 * i + 1]:.4f}   {x[3 * i + 2]:.4f}")

Computed gradients for all possible Double Excitation Gates: 

Excitation : [0, 1, 8, 9], Gradient: -0.10631894572187418
Excitation : [0, 1, 8, 11], Gradient: 0.0
Excitation : [0, 1, 9, 10], Gradient: 0.0
Excitation : [0, 1, 10, 11], Gradient: -0.059240875717551184
Excitation : [0, 2, 8, 10], Gradient: -0.039875832213294854
Excitation : [0, 3, 8, 9], Gradient: 0.0
Excitation : [0, 3, 8, 11], Gradient: 0.0772097850362808
Excitation : [0, 3, 9, 10], Gradient: -0.03733395282298593
Excitation : [0, 3, 10, 11], Gradient: 0.0
Excitation : [0, 4, 8, 10], Gradient: 0.0
Excitation : [0, 5, 8, 9], Gradient: 0.05732746246971731
Excitation : [0, 5, 8, 11], Gradient: 0.0
Excitation : [0, 5, 9, 10], Gradient: 0.0
Excitation : [0, 5, 10, 11], Gradient: -0.014036506114198311
Excitation : [0, 6, 8, 10], Gradient: 0.0
Excitation : [0, 7, 8, 9], Gradient: 0.0
Excitation : [0, 7, 8, 11], Gradient: 0.0
Excitation : [0, 7, 9, 10], Gradient: 0.0
Excitation : [0, 7, 10, 11], Gradient: 0.0
Excitation : [1, 2, 

Step = 12,  E = -74.99046452 Ha,  bond length = 0.90369 Å, bond angle = 91.03941°
Max grad: 0.18576521721702516

Starting step 13:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 13,  E = -74.99317549 Ha,  bond length = 0.91003 Å, bond angle = 90.92424°
Max grad: 0.17860000567404694

Starting step 14:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 14,  E = -74.95544320 Ha,  bond length = 0.91670 Å, bond angle = 90.81695°
Max grad: 4.116210985659221

Starting step 15:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 15,  E = -74.99392223 Ha,  bond length = 0.91628 Å, bond angle = 91.19309°
Max grad: 0.1570644236621109

Starting step 16:
Calculating circuit 

Step = 41,  E = -74.99599093 Ha,  bond length = 0.90935 Å, bond angle = 92.77414°
Max grad: 0.19327511016318588

Starting step 42:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 42,  E = -74.99699169 Ha,  bond length = 0.91171 Å, bond angle = 92.72684°
Max grad: 0.18946283881422427

Starting step 43:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 43,  E = -74.99805072 Ha,  bond length = 0.91424 Å, bond angle = 92.67843°
Max grad: 0.18531342479730684

Starting step 44:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 44,  E = -74.99915543 Ha,  bond length = 0.91692 Å, bond angle = 92.62906°
Max grad: 0.18079318915329878

Starting step 45:
Calculating circu

Step = 70,  E = -75.01136097 Ha,  bond length = 0.94816 Å, bond angle = 94.10840°
Max grad: 0.11739447569184777

Starting step 71:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 71,  E = -75.01152086 Ha,  bond length = 0.94851 Å, bond angle = 94.23143°
Max grad: 0.11623612390666543

Starting step 72:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 72,  E = -75.01172860 Ha,  bond length = 0.94908 Å, bond angle = 94.34228°
Max grad: 0.11439853719320593

Starting step 73:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Finished calculating nuclear coordinates parameters gradient
Step = 73,  E = -75.01197693 Ha,  bond length = 0.94983 Å, bond angle = 94.44198°
Max grad: 0.11195596193596918

Starting step 74:
Calculating circu


KeyboardInterrupt

